In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # Colab 환경설정

!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

In [ ]:
# !pip install matplotlib

In [ ]:
# # github에서 Kobert 파일 로드 및 Kobert 모델 불러오기

# #kobert_tokenizer 폴더 다운받는 코드
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

# #https://github.com/SKTBrain/KoBERT 의 파일들을 다운받는 코드
# !pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master'

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import gluonnlp as nlp

# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
# vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
model_name = "skt/kobert-base-v1"
tokenizer = KoBERTTokenizer.from_pretrained(model_name, use_fast=False)
bertmodel = BertModel.from_pretrained(model_name)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
vocab

In [ ]:
# 필요한 라이브러리 불러오기
# import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import re

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# 입력 데이터셋을 토큰화하기

# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding padding 등을 해주는 코드이다.
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
import re

In [ ]:
def get_clean_text(text):
    result = ""
 
    result = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', text) # 특수문자
    result = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', result) # 한글 자음, 한글 모음
    result = re.sub('([♡❤✌❣♥ᆢ✊❤️✨⤵️☺️;”“]+)', '', result) # 이모티콘 
    only_BMP_pattern = re.compile("["
                        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                       "]+", flags=re.UNICODE)
    result = only_BMP_pattern.sub(r'', result)# BMP characters만
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                               "]+", flags=re.UNICODE)
    result =  emoji_pattern.sub(r'', result) # 유니코드로 이모티콘 지우기
    
    
    return result

In [ ]:
df_train = pd.read_csv("./dataset/train.csv", encoding='utf-8')
df_val = pd.read_csv("./dataset/validation.csv", encoding='utf-8')

In [ ]:
df_train = df_train.replace({"Emotion":{"neutral": 0, "happiness": 1, "sadness": 2, "angry": 3, "disgust": 4, "fear": 5, "surprise": 6}})
df_val = df_val.replace({"Emotion":{"neutral": 0, "happiness": 1, "sadness": 2, "angry": 3, "disgust": 4, "fear": 5, "surprise": 6}})

In [ ]:
for i in tqdm(range(len(df_train))):
    df_train.at[i, '발화문'] = get_clean_text(df_train.at[i, '발화문'])
    
for i in tqdm(range(len(df_val))):
    df_val.at[i, '발화문'] = get_clean_text(df_val.at[i, '발화문'])

In [ ]:
df_train

In [ ]:
df_val

In [ ]:
df_val['Emotion'].value_counts(normalize=True)

In [ ]:
df_val['Emotion'].unique()

In [ ]:
dataset_train = []
for ques, label in zip(df_train['발화문'], df_train['Emotion']):
    data = []
    data.append(ques)
    data.append(label)
    
    dataset_train.append(data)
    
dataset_val = []
for ques, label in zip(df_val['발화문'], df_val['Emotion']):
    data = []
    data.append(ques)
    data.append(label)
    
    dataset_val.append(data)

In [ ]:
dataset_train

In [ ]:
dataset_val

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_validation = BERTDataset(dataset_val,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
validation_dataloader = torch.utils.data.DataLoader(data_validation, batch_size=batch_size, num_workers=0)

In [ ]:
# KoBERT 모델 구현하기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
df_train['Emotion'].value_counts()

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
# 오차행렬, 정확도, 정밀도, 재현율 f1 score을 한번에 출력하기 위한 함수 정의

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_true, y_pred):
#     confusion = confusion_matrix(y_test, pred)
#     answer = torch.cat(y_test).detach().cpu().tolist()
#     temp = torch.cat(pred).detach().cpu().tolist()
#     prediction = []
#     for i in range(len(temp)):
#         prediction.append(temp[i].index(max(temp[i])))
    answer = y_true
    prediction = y_pred
    accuracy = accuracy_score(answer, prediction)
    precision = precision_score(answer, prediction, average= "macro")
    recall = recall_score(answer, prediction, average= "macro")
    f1 = f1_score(answer, prediction, average= "macro")
#     print('Confusion Matrix')
#     print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{} f1 score:{}'.format(accuracy, precision, recall, f1))
    text = '정확도:{}, 정밀도:{}, 재현율:{} f1 score:{}'.format(accuracy, precision, recall, f1)
    return text

In [ ]:
print(len(dataset_train))

In [ ]:
# 학습 모델 저장할 폴더 생성
import os
PATH = f'./Model/batch size={batch_size}, learning rate={learning_rate}, hugging face/'
os.mkdir(PATH)

In [ ]:
# train

In [ ]:
# train

x_true = []
x_pred = []
y_true = []
y_pred = []
train_history=[]
validation_history=[]
loss_history=[]
validation_loss_history=[]
for e in range(num_epochs):
    # epoch별 모델 저장할 폴더 생성
    MODEL_PATH = PATH + f'epoch{e+1}/'
    os.mkdir(MODEL_PATH)
    
#     train_acc = 0.0
#     val_acc = 0.0
    train_acc = 0
    val_acc = 0
    model.train()
    print(f"Epoch {e+1} Train")
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        x_true.extend(label.cpu().detach().numpy().tolist())
        x_pred.extend(torch.argmax(out, dim=1).cpu().detach().tolist())
        # print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
        train_acc = accuracy_score(x_true, x_pred)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc))
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     train_history.append(train_acc / (batch_id+1))
    train_history.append(train_acc)
    loss_history.append(loss.data.cpu().numpy())
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc))
    print()
#     train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    with torch.no_grad():
    #         test_loss = 0
        print(f"Epoch {e+1} Validation")
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(validation_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            y_true.extend(label.cpu().detach().numpy().tolist())    # 정답 y_true 리스트에 cpu로 저장
            out = model(token_ids, valid_length, segment_ids)
#             print(f"y_true : {label}")
#             print(f"y_true : {out}")
            y_pred.extend(torch.argmax(out, dim=1).cpu().detach().tolist())    # 예측값 클래스별 가중치 y_pred 리스트에 cpu로 저장

            loss = loss_fn(out, label)
#             val_acc += calc_accuracy(out, label)
            val_acc = accuracy_score(y_true, y_pred)
            if batch_id % log_interval == 0:
#                 print("epoch {} batch id {} loss {} validation acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), val_acc / (batch_id+1)))
                print("epoch {} batch id {} loss {} validation acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), val_acc))
#         validation_history.append(val_acc / (batch_id+1))
        validation_history.append(val_acc)
        validation_loss_history.append(loss.data.cpu().numpy())

    #         val_loss += loss_fn(out, label).data.cpu().numpy()

#         print("epoch {} validation acc {} ".format(e+1, val_acc / (batch_id+1)))
        print("epoch {} validation acc {} ".format(e+1, val_acc))
    #         validation_history.append(val_acc / (batch_id+1))

        get_clf_eval(y_true, y_pred)

        # 학습 모델 저장
        torch.save(model, MODEL_PATH + 'Model.pt')  # 전체 모델 저장
#         torch.save(model.state_dict(), MODEL_PATH + 'Model_state_dict.pt')  # 모델 객체의 state_dict 저장

        
    

In [ ]:
# 직접 만든 새로운 문장으로 테스트

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("중립이")
            elif np.argmax(logits) == 1:
                test_eval.append("행복이")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 5:
                test_eval.append("공포가")
            elif np.argmax(logits) == 6:
                test_eval.append("놀람이")



        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")